# 目的介绍
* 通过一个运用text来预测是否为骚扰信息的案例，介绍MLlib的pipeline流程
* 介绍HashingTF 和 CountVectorizer的区别

# 数据介绍
这是一个短消息数据，数据用于预测短消息是否存在垃圾消息。输入变量为text，也就是消息文字，输出变量为type，衡量的是否为垃圾信息，0为非垃圾消息，1为垃圾消息。  
数据有5558条，其中747条为垃圾短信。  
本身数据量不是特别大，不到需要用spark的程度，但主要为了熟悉操作使用MLlib写pipeline。

这里有些Spark Pipeline中比较重要的概念，虽然我觉得大部分人都知道，但我感觉还是可以说下，可加深下理解，理清思路。
* DataFrame  
DataFrame是Spark中用于存储信息的一种格式，相比于另外种常见格式 RDD，他多了一个 schema，和传统的格式化数据更相似。
* Transformer  
Transformer是一个Pipelinestage，实现上也是继承自 PipelineStage 类，主要是用来把 一个 DataFrame 转换成另一个 DataFrame。其中比如模型就是一个Transformer。因为模型是将原来的DataFrame append上一列预测的标签，所以是个Transformer。
* Estimator  
Estimator中文翻译为评估器，用于拟合（fit）在 DataFrame上并创造一个Transformer。比如某个算法，如随机森林这就是一个Estimator，通过训练特征数据而得到一个随机森林模型，而这个随机森林模型就是个Transformer。和Transformer区别在于Estimator应用的方法为fit而不是transform，联系在于Estimator运用fit创建Transformer。
* Pipeline  
Pipeline通过将多个Transformer和Estimator串在一起来建立工作流。

Spark Pipeline主要是启发于sklearn的pipeline。这样做有几个好处：
* 降低模型的复杂程度。如果有多个Transformer和Estimator，将会让程序结构复杂，难于理解和实现。
* 便于可以重复利用参数。比如数据预处理步骤很多，可以很方便的用pipeline整合起来，直接保存。还可以避免误操作，比如曾经我把training和testing set都放入PCA，实际上不能这么做。pipeline就可以避免这种事情发生。

# 模型部分

导入需要的包

在Spark的早期版本，sparkContext是进入Spark的切入点。我们都知道RDD是Spark中重要的API，然而它的创建和操作得使用sparkContext提供的API；对于RDD之外的其他东西，我们需要使用其他的Context。比如对于流处理来说，我们得使用StreamingContext；对于SQL得使用sqlContext；而对于hive得使用HiveContext。然而DataSet和Dataframe提供的API逐渐称为新的标准API，我们需要一个切入点来构建它们，所以在 Spark 2.0中我们引入了一个新的切入点(entry point)：SparkSession

In [ ]:
from pyspark import SparkContext
sc = SparkContext()

In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType
from pyspark.sql import SparkSession
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

from pyspark.sql import functions as F

import pandas as pd
import numpy as np

一些设定，在行内作图加上忽略警告

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# 数据读取
主要包括两方面
* 两种数据读取的速度差别
* 将column重命名的两种方法

可以通过csv读取

In [ ]:
%%timeit
spam_data1=sqlContext.read.csv('data/sms_spam_0_1.csv',
                             header='true',
                             inferSchema='true') # spark数据都需要你自己指定schema，单inferschema可以帮助你不用指定schema也可以自动生成对应schema

可以使用parquet格式对数据进行保存和读取  
parquet是一种列式储存格式，可以降低磁盘空间，增加读取速度

In [ ]:
%%timeit
# dataset.write.parquet('data/sms_spam_0_1.parquet2')
spam_data2=spark.read.parquet('data/sms_spam_0_1.parquet2')

这里有个魔术函数，是jupyter notebook内置的一个测量代码时间的魔术函数，叫做timeit。  
但好像加上这个东西执行后，spam_data2不会被保存，所以得自己再读一次数据。

In [ ]:
spam_data = spark.read.parquet('data/sms_spam_0_1.parquet2')

数据读取完毕后查看数据基本情况

In [ ]:
print('here are some examples of data')
spam_data.show(3, truncate = False)
print('number of data in this dataset is {0}, number of spam message is {1} \nschema is shown below'.format(spam_data.count(), spam_data.filter('type = 1').count()))
spam_data.printSchema()

因为spark要求你明确指定一个label，所以将原来的列进行更名，可以用sparksql也可以用DataFrame的withColumnRenamed方法  
个人认为两者之间并没有什么区别，毕竟只是rename一个列

In [ ]:
# 使用SQL前需要注册临时表
spam_data.registerTempTable('whether_spam')#注册临时表
model_data=spark.sql("select type as label,text from whether_spam")
model_data.show(3)

In [ ]:
# spark DataFrame自带一个rename方法
model_data = spam_data.withColumnRenamed("type" , "label")
model_data.show(3)

# 具体数据

In [ ]:
model_data.filter("label = 0").show(20, truncate= False)

In [ ]:
model_data.filter("label = 1").show(20, truncate= False)

可以看到普通短信都挺普通的，但垃圾邮件几乎都有字眼如award, win, sex等字眼。

# 划分训练集和测试集

In [ ]:
train, test = model_data.randomSplit([0.80, 0.20], seed=42)#划分训练集和测试集,设置seed

In [ ]:
print (train.count(),test.count())

# 建立机器学习pipeline
机器学习部分主要是想通过将文本转换成TF-IDF然后进行预测

In [ ]:
from pyspark.ml.feature import RegexTokenizer,CountVectorizer,IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder

我们想建立一个pipeline  
所以会在下一个cell中一并设置好我们想使用的函数和其使用的参数。  
第一步是将语句转换为词袋，使用的是RegexTokenizer。RegexTokenizer基于正则表达式匹配提供了更高级的断词(tokenization)。默认情况下,参数pattern(默认是\s+)（即空白）作为分隔符。  
第二步是统计中文本的数量，使用的是CountVectorizer，作用是将文本向量转换成词频向量。  
第三步是想将上述转换成TF-IDF。  

为了更清楚说明，我们将这个pipeline拆开。

In [ ]:
print('original DataFrame:')
train.show(5)

In [ ]:
tokenizer=RegexTokenizer(inputCol='text',outputCol='words',pattern='[^a-zA-Z]')
wordsData = tokenizer.transform(train)
print('After tokenize:')
wordsData.show(5)

cv=CountVectorizer(inputCol='words',outputCol='tf')
cvmodel = cv.fit(wordsData)
featurizedData = cvmodel.transform(wordsData)
print('After CountVectorizer:')
featurizedData.show(5)

idf=IDF(inputCol='tf',outputCol='features')#IDF
idfmodel = idf.fit(featurizedData)
rescaledData = idfmodel.transform(featurizedData)
print('After IDF:')
rescaledData.show(5)

print('Only show label and features:')
rescaledData.select("label", "features").show(3)

In [ ]:
# 需注意tf和feature不一样
rescaledData.select("tf","features").show(1, truncate= False)

做TF-IDF其实spark中更常见的是HashingTF。他们俩的联系在于作用都是创造一个词频向量，区别在于他们俩创造词频向量的方法不一样。

CountVectorizer会首先创造一个单词表，将所有涉及到的单词来创造这个单词表，并且在创建这个单词表的时候，他会默默记录下哪些单词出现的次数多，哪些单词出现的次数少。  
这在待会后面的调参部分，有个参数叫做vocabSize，这个参数用于收缩单词表，比如只要3000个单词构成词频向量。最终他会创建一个稀疏向量用于之后的处理。但他总归要计算一个全局的词汇表，所以运算花费会更大  
他创建的这个feature第一个部分表示单词表有多大，第二个部分表示这个单词的索引多少，第三个部分表示这个单词出现的次数。

HashingTF也是转换为词频向量的一个方法。说起HashingTF就得说FeatureHasing或者叫做HashingTrick。这两者简单来说就是使用hash方法对离散数据进行降维。默认使用的是MurmurHash 3。据说有研究表明HashingTF不会降低模型效果。

这两者的优缺点为
* HashingTF后的东西完全没有任何解释性了，虽然一般其实也不大会要求CountVectorizer有什么解释性
* HashingTF所用时间更合理
* HashingTF会存在collusion，即把多个不同的单词Hash在一起。这样可能会降低模型效果。比如有的字眼虽然少，但预测力很强，结果HashingTF直接给Hashing在一起了。
* CountVectorizer可以对转换后的数据进行还原，而HashingTF不行

HashingTF有一个参数叫做numFeatures，用于设定最高的Feature数量，默认为2的20次方。但我们需注意，HashingTF的collusion是不可避免的。比如一共只有30个单词，我们映射到一个32维的空间内，非常大概率会出现HashingCollusion。除此之外，spark官方文档写到，如果希望能够尽量平均的Hashing，numFeatures尽量选用2的幂。

In [ ]:
%%timeit
cv=CountVectorizer(inputCol='words',outputCol='tf')
cvmodel = cv.fit(wordsData)
featurizedData = cvmodel.transform(wordsData)

In [ ]:
from pyspark.ml.feature import HashingTF

In [ ]:
%%timeit
hashingTF = HashingTF(inputCol='words',outputCol='tf')
hasingData = hashingTF.transform(wordsData)

In [ ]:
hashingTF = HashingTF(inputCol='words',outputCol='tf', numFeatures = 8192)
hasingData = hashingTF.transform(wordsData)
hasingData.select('tf').show(5, truncate = False)

把上述步骤（CountVectorizer）加上逻辑回归模型打包成一个pipeline

In [ ]:
tokenizer=RegexTokenizer(inputCol='text',outputCol='words',pattern='[^a-zA-Z]')# 除了字母以外都当成断词的标准
cv=CountVectorizer(inputCol='words',outputCol='tf')#TF
idf=IDF(inputCol='tf',outputCol='features')#IDF
lr=LogisticRegression(maxIter=30)#logistic
pipeline=Pipeline(stages=[tokenizer,cv,idf,lr])#pipline通道

# 调参

设置好调参部分和评价标准，  
标准设置为BinaryClassificationEvaluator，areaUnderROC是其默认的评价指标。  
调参的几个部分分别为regParam正则化程度，elasticNetParam设置L1L2正则的比例，vocabSize设置的是CountVec中最大的单词的数量，是一个计算速度和效果的权衡，minDocFreq用于声明词语最少出现多少次。

In [ ]:
auc_eval = BinaryClassificationEvaluator()#评价方法
grid = ParamGridBuilder().addGrid(lr.regParam,[1e-3,1e-2,1e-1]).addGrid(lr.elasticNetParam,[0.25,0.5,0.75]).addGrid(cv.vocabSize,[3000,5000,7000,9000]).addGrid(idf.minDocFreq,[0,3])\
    .build()#为每个stage定义参数

In [ ]:
cross_val=CrossValidator(estimator=pipeline,\
                         evaluator=auc_eval,\
                         estimatorParamMaps=grid,numFolds=10)#10折交叉验证
pipeline_model=cross_val.fit(train)#根据各种参数运行模型

可以看到有哪些参数

In [ ]:
# 0号模型模型各类参数
grid[0]

In [ ]:
# 模型效果
pipeline_model.avgMetrics[:]

In [ ]:
scores=list(zip(grid,pipeline_model.avgMetrics))#将模型参数和模型效果值合成元组
scores.sort(key=lambda x:x[1],reverse=True)#按照模型得分倒序排序

In [ ]:
print ('cross-validation scores:')
for p,s in scores:
    print ('regParam:  %s; elasticNet: %s:,vocabSize: %s,minDocFreq: %s,ROC_score:%s'%(p[lr.regParam],p[lr.elasticNetParam],\
                                                                                     p[cv.vocabSize],p[idf.minDocFreq],s))

PipeLine有些自带的method，这里用到getStage来显示有哪些阶段

In [ ]:
print(pipeline.getStages())#通道中的内容

In [ ]:
pipeline_model是一个CrossValidatorModel，他包含两个部分，第一个部分记录拥有最好的metric的模型，另外一部分同时跟踪了其他模型的表现。对于第一点，我们可以在稍后看到我们可以直接用pipeline_model来transform DataFrame。第二点我们可以在之前的例子内看到CrossValidatorModel保存有各个模型的avgMetrics

In [ ]:
print(pipeline_model)

In [ ]:
print(pipeline_model.bestModel)

In [ ]:
train_transform=pipeline_model.transform(train)
train_transform.toPandas().head()

可能欺诈邮件的预测比较有意思，看一下预测错的邮件长什么样

In [ ]:
spam_trainset = model_data.filter("label = 1")
spam_transform = pipeline_model.transform(spam_trainset)
TP = spam_transform.filter('prediction = 1').count()
FN = spam_transform.filter('prediction = 0').count()
print('True positive is {0}, False negative is {1}. And following are some false negative'.format(TP, FN))
spam_transform.filter('prediction = 0').select('text').show(10, truncate= False)

# 预测测试集数据

In [ ]:
test_predict = pipeline_model.transform(test)

In [ ]:
auc_eval.evaluate(test_predict)